In [119]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import time
options = {
 "WLSACCESSID":"a4353fb7-f95b-4075-b288-ca3f60983b36",
"WLSSECRET":"d894d460-2dac-4210-8c40-c91c68ecfb13",
"LICENSEID":2562382
}

In [120]:
K = 4
lambda_k_true = np.array([1,15,-1,1])

I = 100
J = 100



### generate exogenous data
np.random.seed(1)

φ_i_j_k = np.random.normal(1,1, size=[I,J,K-1])

def φ_k(i,B):
    # return φ_i_j_k[i,B,:].sum(0)
    return np.concatenate((φ_i_j_k[i,B,:].sum(0), [ - np.sum(B)**(2)]))

### greedy
def greedy(i,lambda_k, p_j = np.zeros(J)):
    B_k =  np.zeros(J, dtype=bool)
    val = np.inner(φ_k(i,B_k),lambda_k)
    k = 0 
    while k < J:
        j_k = None
        max_add_j =  - np.inf
        for j in np.where(1-B_k)[0]:
            val_add_j = np.inner(φ_k(i,B_k+ np.eye(1,J,j,dtype=bool)[0]), lambda_k)   - p_j[j] - p_j[B_k].sum() - val
            if val_add_j > max_add_j:
                j_k = j
                max_add_j = val_add_j
            
        if j_k is None or max_add_j < 0:
            break
        val += max_add_j 
        B_k[j_k] = 1
        k += 1
    return B_k , val

In [121]:
greedy(0,lambda_k_true)

(array([False, False, False,  True, False, False, False, False, False,
        False, False, False, False, False,  True, False, False, False,
        False, False,  True, False,  True, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False,  True, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False,  True, False, False, False, False, False,
         True,  True, False,  True, False, False, False, False, False,
        False,  True, False, False, False, False, False, False, False,
        False, False, False,  True,  True, False, False,  True, False,
         True, False,  True, False, False, False, False, False, False,
        False]),
 330.55051573566146)

$\begin{align}
\min_{u,p}& \quad \sum_{i} u_{i} + \sum_{j} p_j \notag\\
s.t.& \quad u_{i} + \sum_{j \in B}p_j \geq \phi_{iB}^\top\lambda\quad \text{ for all }iB \notag
\end{align}$

In [122]:
max_iters = 1000
tol = 1e-12

constraints_list = []

# Create the environment with license parameters
with gp.Env(params=options) as env:
    # Create a Gurobi model within the environment
    with gp.Model(env=env) as model:
        ### Initialize 
        # Create variables
        u_i = model.addVars(I, name="utilities")
        # p_j = model.addVars(J, name="prices")
        p_j = model.addVars(J, lb= 0 ,ub=GRB.INFINITY, name="prices")

        # Set objective
        model.setObjective(u_i.sum() + p_j.sum(), GRB.MINIMIZE)
        
        # Optimize the model
        model.setParam('OutputFlag', 0)
        model.optimize()
        
        # Extract the solution (theta_solution)
        theta_solution = np.array(model.x)

        
        ### Column Generation
        iter = 0
        while iter < max_iters:
            print(f"ITER: {iter}")
            ### Pricing problem
            start_time = time.time()
            B_star_i = []
            reduced_cost_i = []
            for i in range(I):
                        B_star, val = greedy(i  ,lambda_k = lambda_k_true, p_j = theta_solution[-J:])
                        B_star_i.append(B_star)
                        reduced_cost_i.append(val -  theta_solution[i])
            end_time = time.time()
            print(f"greedy done:{end_time - start_time}")
            # stop if certificate holds
            print(f"reduced cost: {np.max(reduced_cost_i)}")
            start_time = time.time()
            if np.max(reduced_cost_i) <= tol:
                primal_solution = np.array(model.x)
                dual_solution = np.array(model.pi)
                break
            
            ### Master problem
            model.addConstrs((u_i[i] + gp.quicksum(p_j[j] for j in np.where(B_star_i[i])[0])>= 
                            φ_k(i, B_star_i[i])@lambda_k_true 
                            for i in range(I)), name="constraint_batch")
            
            constraints_list.append(B_star_i)
            end_time = time.time()
            # print(f"constraints added:{end_time - start_time}")

            # Optimize the model
            start_time = time.time()
            u_i.start = reduced_cost_i + theta_solution[:I]
            p_j.start = theta_solution[-J:]
            model.optimize()
            theta_solution = np.array(model.x)
            iter += 1
            end_time = time.time()  
            # print(f"model solved:{end_time - start_time}")
            print("##############")
            

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2562382
Academic license 2562382 - for non-commercial use only - registered to ed___@nyu.edu
ITER: 0
greedy done:1.1544227600097656
reduced cost: 472.6855059438528
##############
ITER: 1
greedy done:0.9200241565704346
reduced cost: 322.15551860316526
##############
ITER: 2
greedy done:0.8311948776245117
reduced cost: 282.4386719050864
##############
ITER: 3
greedy done:0.7615711688995361
reduced cost: 239.0360602727145
##############
ITER: 4
greedy done:0.691504955291748
reduced cost: 222.4717462958747
##############
ITER: 5
greedy done:0.6336309909820557
reduced cost: 227.0311260286187
##############
ITER: 6
greedy done:0.6480169296264648
reduced cost: 226.06786079383335
##############
ITER: 7
greedy done:0.5529358386993408
reduced cost: 171.34826079243027
##############
ITER: 8
greedy done:0.5258607864379883
reduced cost: 141.71063464954972
##############
ITER: 9
greedy done:0.5149030685424805
reduced 

In [117]:
binding_constraints = np.array(constraints_list).reshape(-1,J)[dual_solution >0]
i = np.kron(np.ones(iter, dtype= int),np.arange(I))
matching = np.zeros((I ,J), dtype=bool)
matching[i[dual_solution > 0]]= binding_constraints

In [118]:
matching.sum(0)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [101]:
primal_solution[K-1: I+K-1]

array([34.90491836, 19.87810364, 52.36259099, 21.65322423, 39.67809581,
        3.40661492, 13.760347  , 12.74384418, 21.66703773, 10.69149945,
       20.74664842, 20.15436801, 13.62325182, 12.        ,  6.11610998,
       19.0727004 ,  2.        , 18.3147408 , 36.13941265, 11.9728226 ,
        5.3137028 ,  8.64397208,  6.        , 17.23929345, 37.26092185,
       36.57949941, 13.88907165, 28.70309716, 50.53613869, 42.67566957])

In [102]:
B_star_i = []
val_i = []
for i in range(I):
        B_star, val = greedy(i ,lambda_k = lambda_k_true, p_j = theta_solution[-J:])
        B_star_i.append(B_star)
        val_i.append(val -  theta_solution[i])
print(np.max(val_i))

2.842170943040401e-14


In [103]:
sample = matching
m_j = sample.sum(0)
phi_k_hat = np.zeros(K)
for i in range(I):
    phi_k_hat += φ_k(i,sample[i,:])
print(phi_k_hat)

[ 107.85562007  303.08593775   74.43202429 -360.        ]


In [104]:
sample.sum(0)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

$\begin{align}
\max_{\lambda ,u,p}& \quad \sum_{ik} \phi_{i\hat B_i,k} \lambda_k - \sum_{i} u_{i} -  \sum_{j} p_j \notag\\
s.t.& \quad u_{i} + \sum_{j \in B}p_j \geq \sum_{k} \phi_{i B,k} \lambda_k + \phi_{i B,0} \quad \text{ for all }iB \notag
\end{align}$

In [105]:
max_iters = 1000
tol = 1e-12

with gp.Env(params=options) as env:
    with gp.Model(env=env) as model:
        ### Initialize 
        # Create variables
        lambda_k = model.addVars(K-1, lb= -GRB.INFINITY, ub = GRB.INFINITY , name="parameters")
        u_i = model.addVars(I, name="utilities")
        p_j = model.addVars(J,  lb= -GRB.INFINITY, ub = GRB.INFINITY, name="prices")
        # p_j = model.addVars(J, name="prices")

        # Set objective
        model.setObjective(gp.quicksum(phi_k_hat[k+1] * lambda_k[k] for k in range(K-1))
                            - u_i.sum() -  gp.quicksum(m_j[j] * p_j[j] for j in range(J)), GRB.MAXIMIZE)
        
        model.addConstrs((u_i[i] + gp.quicksum(p_j[j] for j in np.where(sample[i])[0])>= 
                            gp.quicksum(φ_k(i,sample[i])[k+1] *  lambda_k[k] for k in range(K-1)) + φ_k(i,sample[i])[0]
                            for i in range(I)), name="constraint_batch")
        
        # Optimize the model
        model.setParam('OutputFlag', 0)
        # model.setParam('Presolve', 0)
        model.optimize()
        status = model.Status
        
        # Extract the solution (theta_solution)
        theta_solution = np.array(model.x)

        ### Column Generation
        iter = 0
        while iter < max_iters:

            # print(f"ITER: {iter}")
            # print(theta_solution[:K-1])
            ### Pricing problem
            start_time = time.time()
            B_star_i = []
            reduced_cost_i = []
            for i in range(I):
                B_star, val = greedy(i, lambda_k = np.concatenate( ([1],theta_solution[:K-1])), p_j = theta_solution[-J:])
                B_star_i.append(B_star)
                reduced_cost_i.append(val -  theta_solution[K-1+i])
            end_time = time.time()
            # print(f"greedy done:{end_time - start_time}")
            # stop if certificate holds
            print(f"reduced cost: {np.max(reduced_cost_i)}")
            start_time = time.time()
            if np.max(reduced_cost_i) <= tol:
                primal_solutin = np.array(model.x)
                dual_solution = np.array(model.pi)
                print("done!")
                break
            
            ### Master problem
            model.addConstrs((u_i[i] + gp.quicksum(p_j[j] for j in np.where(B_star_i[i])[0])>= 
                            gp.quicksum(φ_k(i, B_star_i[i])[k+1] *  lambda_k[k] for k in range(K-1))+ 
                            φ_k(i, B_star_i[i])[0]
                            for i in range(I)), name="constraint_batch")
            
            # print(f"constraints added:{end_time - start_time}")

            # Optimize the model
            lambda_k.start = theta_solution[:K-1]
            u_i.start = reduced_cost_i + theta_solution[K-1:I +K-1]
            p_j.start = theta_solution[-J:]
            model.optimize()
            theta_solution = np.array(model.x)
            iter += 1
            # print("##############")
            

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2562382
Academic license 2562382 - for non-commercial use only - registered to ed___@nyu.edu
reduced cost: 95.74256169355878
reduced cost: 427.8710061616923
reduced cost: 308.0968943590546
reduced cost: 445.63567954832087
reduced cost: 280.38300010253147
reduced cost: 264.29446331403
reduced cost: 209.84826136638523
reduced cost: 176.330612037954
reduced cost: 211.5885993143905
reduced cost: 150.1250091269356
reduced cost: 190.56092340839623
reduced cost: 216.01296400410234
reduced cost: 151.37490639342482
reduced cost: 129.31193375124897
reduced cost: 123.89510699131264
reduced cost: 137.8798871300282
reduced cost: 116.05323722066126
reduced cost: 107.66708125834424
reduced cost: 103.4978244510404
reduced cost: 102.11731648524244
reduced cost: 97.92697240654286
reduced cost: 68.18355482460105
reduced cost: 94.51289591790439
reduced cost: 93.1786209959305
reduced cost: 90.80726921032127
reduced cost: 85.

In [106]:
print(theta_solution[:K-1])
print(lambda_k_true[1:])

[13.59860058 -1.00435953  0.74282829]
[15 -1  1]


In [107]:
m_j

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])